## Load data

In [7]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf

# all_files_model1 = ['airplane', 'ant', 'axe', 'banana', 'bicycle', 'bridge', 'butterfly', 'castle', 'cat', 'chair', 'diamond', 'donut', 'The_Eiffel_Tower']
all_files = ['apple', 'banana', 'cup', 'finger', 'fish', 'key', 'smiley_face', 'sock', 'sword']
i = 0
while i < len(all_files):
    all_files[i] = "full_numpy_bitmap_" + all_files[i] + ".npy"
    i +=1 

max_items_per_class = 7500

x = np.empty([0, 784])
y = np.empty([0])
class_names = []

#load a subset of the data to memory 
for idx, file in enumerate(all_files):
    data = np.load("Images/" + file)
    data = data[0: max_items_per_class, :]
    labels = np.full(data.shape[0], idx)
    x = np.concatenate((x, data), axis=0)
    y = np.append(y, labels)
    class_name, ext = os.path.splitext(os.path.basename(file))
    class_names.append(class_name.replace('full_numpy_bitmap_', ''))

#separate into training and testing 
permutation = np.random.permutation(y.shape[0])
x = x[permutation, :]
y = y[permutation]

vfold_size = int(x.shape[0]/100*(0.2*100))

x_test = x[0:vfold_size, :]
y_test = y[0:vfold_size]

x_train = x[vfold_size:x.shape[0], :]
y_train = y[vfold_size:y.shape[0]]

## Preprocess the data

In [8]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, len(class_names))
y_test = keras.utils.to_categorical(y_test, len(class_names))

## Build the model

In [9]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(len(class_names), activation='softmax')) 
# Train model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['top_k_categorical_accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 3, 3, 64)         

In [10]:
#fit the model 
model.fit(x_train, y_train, batch_size = 256, verbose=2, epochs=10)

#evaluate on unseeFor prediction we use model.predict this will return probabilities of the shape [N, 100n data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: {:0.2f}%'.format(score[1] * 100))

Epoch 1/10
563/563 - 34s - loss: 0.5071 - top_k_categorical_accuracy: 0.9751 - 34s/epoch - 60ms/step
Epoch 2/10
563/563 - 32s - loss: 0.2715 - top_k_categorical_accuracy: 0.9876 - 32s/epoch - 57ms/step
Epoch 3/10
563/563 - 31s - loss: 0.2257 - top_k_categorical_accuracy: 0.9899 - 31s/epoch - 54ms/step
Epoch 4/10
563/563 - 34s - loss: 0.1983 - top_k_categorical_accuracy: 0.9915 - 34s/epoch - 60ms/step
Epoch 5/10
563/563 - 30s - loss: 0.1808 - top_k_categorical_accuracy: 0.9922 - 30s/epoch - 54ms/step
Epoch 6/10
563/563 - 30s - loss: 0.1646 - top_k_categorical_accuracy: 0.9930 - 30s/epoch - 53ms/step
Epoch 7/10
563/563 - 32s - loss: 0.1540 - top_k_categorical_accuracy: 0.9937 - 32s/epoch - 56ms/step
Epoch 8/10
563/563 - 33s - loss: 0.1431 - top_k_categorical_accuracy: 0.9945 - 33s/epoch - 58ms/step
Epoch 9/10
563/563 - 33s - loss: 0.1346 - top_k_categorical_accuracy: 0.9948 - 33s/epoch - 59ms/step
Epoch 10/10
563/563 - 31s - loss: 0.1261 - top_k_categorical_accuracy: 0.9955 - 31s/epoch -

In [13]:
model.save('pretrained_model2.h5')